In [ ]:
!pip install --upgrade azureml-core==1.30.0

In [2]:
import azureml.core


print(f"Azure ML SDK Version: {azureml.core.VERSION}")

Azure ML SDK Version: 1.30.0


In [1]:
from azureml.core import Workspace


ws = Workspace.from_config()
ws

Workspace.create(name='wshandsonaml', subscription_id='23376daf-77f3-4195-9649-e223b8072ad9', resource_group='rg-handsonaml')

### Key Vault

In [2]:
account_key = "CECI N'EST PAS UNE BONNE PRATIQUE"

In [3]:
from azureml.core import Keyvault

keyvault = ws.get_default_keyvault()

#On préfèrera saisir les informations manuellements dans le Key Vault plutôt que par le code
keyvault.set_secret(name="sahandsonaml-account-key", value = account_key)

#Attention, il est possible de voir en clair les secrets obtenus avec la méthode get_secret
keyvault.get_secret(name="sahandsonaml-account-key")

"CECI N'EST PAS UNE BONNE PRATIQUE"

### Datastore

In [4]:
# default datastore
from azureml.core import Datastore

Datastore.get_default(ws)

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-1e460512-3e9b-4006-b0af-cc450b8c1361",
  "account_name": "sahandsonaml",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [62]:
datastore_name = "mydsstore"
container_name = "data"
account_name = "sahandsonaml"

In [63]:
from azureml.core import Datastore

# create data store (Blob storage)
Datastore.register_azure_blob_container(ws,
                                        datastore_name=datastore_name,
                                        container_name=container_name,
                                        account_name=account_name,
                                        sas_token=None,
                                        account_key=keyvault.get_secret(name="sahandsonaml-acc-key"),
                                        protocol=None,
                                        endpoint=None,
                                        overwrite=False,
                                        create_if_not_exists=False,
                                        skip_validation=False,
                                        blob_cache_timeout=None,
                                        grant_workspace_access=False,
                                        subscription_id=None,
                                        resource_group=None)


{
  "name": "mydsstore",
  "container_name": "data",
  "account_name": "sahandsonaml",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [64]:
# List all the datastores
for ds_name in ws.datastores:
        print(ds_name)

mydsstore
workspacefilestore
workspaceblobstore


In [65]:
mydsstore = Datastore.get(ws, datastore_name)
mydsstore

{
  "name": "mydsstore",
  "container_name": "data",
  "account_name": "sahandsonaml",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [12]:
mydsstore.set_as_default()

In [61]:
mydsstore.unregister()

In [14]:
# List all the datastores
for ds_name in ws.datastores:
        print(ds_name)

workspacefilestore
workspaceblobstore


### Dataset

In [68]:
mydsstore = Datastore.get(ws, "workspaceblobstore")

In [69]:
mydsstore.upload(src_dir='./data', target_path='up/', overwrite=True)

Uploading an estimated of 2 files
Uploading ./data/german_credit_full.csv
Uploaded ./data/german_credit_full.csv, 1 files out of an estimated total of 2
Uploading ./data/german_credit_init.csv
Uploaded ./data/german_credit_init.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_102da256e7f84779be8101ac8c9b2040

In [70]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath

dataset = Dataset.Tabular.from_delimited_files(path = DataPath(mydsstore, 'up/german_credit_init.csv'))

In [71]:
# Convert the Dataset object to a (in-memory) pandas dataframe
df = dataset.to_pandas_dataframe()

In [72]:
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,critical/other existing credit,radio/tv,1169,no known savings,>=7,4,male single,none,...,real estate,67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,existing paid,radio/tv,5951,<100,1<=X<4,2,female div/dep/mar,none,...,real estate,22,none,own,1,skilled,1,none,yes,bad
2,no checking,12,critical/other existing credit,education,2096,<100,4<=X<7,2,male single,none,...,real estate,49,none,own,1,unskilled resident,2,none,yes,good
3,<0,42,existing paid,furniture/equipment,7882,<100,4<=X<7,2,male single,guarantor,...,life insurance,45,none,for free,1,skilled,2,none,yes,good
4,<0,24,delayed previously,new car,4870,<100,1<=X<4,3,male single,none,...,no known property,53,none,for free,2,skilled,2,none,yes,bad


In [73]:
df.count()

checking_status           100
duration                  100
credit_history            100
purpose                   100
credit_amount             100
savings_status            100
employment                100
installment_commitment    100
personal_status           100
other_parties             100
residence_since           100
property_magnitude        100
age                       100
other_payment_plans       100
housing                   100
existing_credits          100
job                       100
num_dependents            100
own_telephone             100
foreign_worker            100
class                     100
dtype: int64

In [74]:
dataset.register(ws, "german_credit_dataset")

UserErrorException: UserErrorException:
	Message: There is already a dataset registered under name "german_credit_dataset". Specify `create_new_version=True` to register the dataset as a new version. Use `update`, `add_tags`, or `remove_tags` to change only the description or tags.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "There is already a dataset registered under name \"german_credit_dataset\". Specify `create_new_version=True` to register the dataset as a new version. Use `update`, `add_tags`, or `remove_tags` to change only the description or tags."
    }
}

In [75]:
dataset.version

In [78]:
# mettre à jour le dataset
dataset = Dataset.Tabular.from_delimited_files(path = DataPath(mydsstore, 'up/german_credit_full.csv'))
dataset.register(ws, "german_credit_dataset", create_new_version=True)
print(dataset.version)

None
